<a href="https://colab.research.google.com/github/jeonseungho-glitch/Road_Lane_segmentation/blob/main/Autosnomous_steering_wheel_with_Seg%2Blanedetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오버레이 및 시각화 원본이미지 주행차선 그리기 -> Seg이미지 오버레이

In [19]:
# ============================================================
# Final Integrated Code: Dual Input (PNG/JPG) + Stabilized Steering
# Fixed: ValueError on Mutable Defaults
# ============================================================

!pip -q install -U segmentation-models-pytorch timm natsort

import os
import glob
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from natsort import natsorted
import segmentation_models_pytorch as smp
from dataclasses import dataclass
from typing import Tuple, Optional, List

# ==========================================
# 1) Configuration (Centralized Control)
# ==========================================
@dataclass(frozen=True)
class Config:
    # --- Paths ---
    CKPT_PATH: str      = "/content/drive/MyDrive/DataSet/best.pt"                       # Lane Detection Model
    RAW_IMG_DIR: str    = "/content/drive/MyDrive/DataSet/sample/inference_img"          # Original Image Set
    SEG_VIS_DIR: str    = "/content/dataset/inference/sample_img"                        # Segmentation Image Set
    WHEEL_PATH: str     = "/content/drive/MyDrive/DataSet/Yoke_Steering_wheel.png"       # Steering Wheel Image
    OUT_VIDEO_PATH: str = "/content/drive/MyDrive/DataSet/Autonomous_Steering.mp4"       # Out Video Path & Name

    # --- Video & Model Settings ---
    FPS: int            = 10
    ENCODER_NAME: str   = "resnet50"
    NUM_CLASSES: int    = 1
    RESIZE_SHORTEST: int= 1024
    RESIZE_LONGEST: int = 1920
    SIZE_DIVISOR: int   = 32
    THRESH: float       = 0.5

    # --- Visualization ---
    LANE_COLOR_BGR: Tuple[int, int, int] = (0, 255, 0)
    LANE_ALPHA: float   = 0.45
    SCAN_ALPHA: float   = 0.60
    SCANLINE_COLOR: Tuple[int, int, int] = (255, 0, 0) # Blue

    # --- Steering & Control Logic ---
    SCAN_Y_RATIO: float         = 0.70  # Scanline height ratio
    STEERING_SENSITIVITY: float = 3.0   # Multiplier
    SMOOTHING: float            = 0.15  # Exponential Moving Average factor
    POWER_FACTOR: float         = 1.4   # Non-linear response
    DEADZONE: float             = 1.9   # Minimum angle to react
    LANE_OFFSET_PIXELS: int     = 160   # Virtual lane center offset
    MAX_ANGLE: float            = 90.0  # Physical limit

    # --- System ---
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

    # [FIX] Use Tuples instead of np.array to avoid ValueError in dataclass
    # Numpy will automatically handle the math with these tuples.
    MEAN: Tuple[float, float, float] = (0.485, 0.456, 0.406)
    STD: Tuple[float, float, float]  = (0.229, 0.224, 0.225)

# ==========================================
# 2) Image Processing Utilities
# ==========================================
class ImageUtils:
    @staticmethod
    def resize_keep_aspect(H: int, W: int, shortest: int, longest: int) -> Tuple[int, int]:
        scale = shortest / min(H, W)
        newH, newW = int(round(H * scale)), int(round(W * scale))
        if max(newH, newW) > longest:
            scale = longest / max(newH, newW)
            newH, newW = int(round(newH * scale)), int(round(newW * scale))
        return max(1, newH), max(1, newW)

    @staticmethod
    def pad_to_divisor_chw(x_chw: torch.Tensor, divisor=32, pad_value=0.0):
        C, H, W = x_chw.shape
        padH = (divisor - H % divisor) % divisor
        padW = (divisor - W % divisor) % divisor
        if padH == 0 and padW == 0:
            return x_chw, (0, 0, 0, 0)
        x_chw = F.pad(x_chw, (0, padW, 0, padH), value=float(pad_value))
        return x_chw, (0, padW, 0, padH)

    @staticmethod
    def preprocess(frame_bgr: np.ndarray) -> Tuple[torch.Tensor, Tuple[int, int], Tuple[int, int]]:
        origH, origW = frame_bgr.shape[:2]
        newH, newW = ImageUtils.resize_keep_aspect(origH, origW, Config.RESIZE_SHORTEST, Config.RESIZE_LONGEST)

        rs = cv2.resize(frame_bgr, (newW, newH), interpolation=cv2.INTER_LINEAR)
        rgb = cv2.cvtColor(rs, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

        # [NOTE] Numpy broadcasts tuples (Config.MEAN) against arrays automatically
        rgb = (rgb - Config.MEAN) / Config.STD

        x = torch.from_numpy(rgb).permute(2, 0, 1).float()
        x, _ = ImageUtils.pad_to_divisor_chw(x, divisor=Config.SIZE_DIVISOR, pad_value=0.0)
        x = x.unsqueeze(0)
        return x, (origH, origW), (newH, newW)

# ==========================================
# 3) Lane Pilot Class (Logic & Visualization)
# ==========================================
class LanePilot:
    def __init__(self):
        self.model = self._load_model()
        self.prev_angle = 0.0
        self.wheel_img = self._load_wheel()

        # Performance settings
        cv2.setNumThreads(0)
        torch.backends.cuda.matmul.allow_tf32 = True
        try:
            torch.set_float32_matmul_precision("high")
        except:
            pass

    def _load_model(self):
        print(f"[Init] Loading model from {Config.CKPT_PATH}...")
        ckpt = torch.load(Config.CKPT_PATH, map_location="cpu")
        sd = ckpt["model_state"] if isinstance(ckpt, dict) and "model_state" in ckpt else ckpt

        # Key normalization
        clean_sd = {}
        for k, v in sd.items():
            for p in ["module.", "model.", "_orig_mod.", "orig_mod."]:
                if k.startswith(p): k = k[len(p):]
            k = k.replace("module._orig_mod.", "").replace("model._orig_mod.", "")
            clean_sd[k] = v

        model = smp.DeepLabV3Plus(
            encoder_name=Config.ENCODER_NAME,
            encoder_weights=None,
            in_channels=3,
            classes=Config.NUM_CLASSES,
            activation=None,
        )
        model.load_state_dict(clean_sd, strict=False)
        model.to(Config.DEVICE).eval()
        return model

    def _load_wheel(self):
        if not os.path.exists(Config.WHEEL_PATH):
            raise FileNotFoundError(f"Wheel image not found at {Config.WHEEL_PATH}")
        return cv2.imread(Config.WHEEL_PATH, cv2.IMREAD_UNCHANGED)

    @torch.no_grad()
    def infer_mask(self, frame_bgr: np.ndarray) -> np.ndarray:
        x, (origH, origW), (newH, newW) = ImageUtils.preprocess(frame_bgr)
        x = x.to(Config.DEVICE, non_blocking=True)

        with torch.autocast(device_type="cuda", dtype=torch.bfloat16, enabled=(Config.DEVICE == "cuda")):
            logits = self.model(x)

        if isinstance(logits, (list, tuple)): logits = logits[0]
        if isinstance(logits, dict): logits = logits.get("logits", logits.get("out"))

        logits = logits[:, :, :newH, :newW]
        logits_up = F.interpolate(logits, size=(origH, origW), mode="bilinear", align_corners=False)
        prob = torch.sigmoid(logits_up)[0, 0].detach().cpu().numpy()
        return (prob >= Config.THRESH).astype(np.uint8)

    def calculate_steering(self, mask_pred: np.ndarray) -> Tuple[float, int, int]:
        h, w = mask_pred.shape
        img_center = w // 2
        scan_y = int(h * Config.SCAN_Y_RATIO)

        x_indices = np.where(mask_pred[scan_y, :] > 0)[0]

        if len(x_indices) == 0:
            return 0.0, img_center, scan_y

        left_indices = x_indices[x_indices < img_center]
        right_indices = x_indices[x_indices >= img_center]

        l_inner = np.max(left_indices) if len(left_indices) > 0 else None
        r_inner = np.min(right_indices) if len(right_indices) > 0 else None

        if l_inner is not None and r_inner is not None:
            lane_center = (l_inner + r_inner) // 2
        elif l_inner is not None:
            lane_center = l_inner + Config.LANE_OFFSET_PIXELS
        elif r_inner is not None:
            lane_center = r_inner - Config.LANE_OFFSET_PIXELS
        else:
            return 0.0, img_center, scan_y # Lane lost

        # --- Control Logic ---
        offset = lane_center - img_center
        normalized_offset = offset / (w / 2) # -1.0 to 1.0

        # Non-linear steering response
        angle = np.sign(offset) * (abs(normalized_offset) ** Config.POWER_FACTOR) * 90 * Config.STEERING_SENSITIVITY

        # Deadzone
        if abs(angle) < Config.DEADZONE:
            angle = 0.0

        angle = np.clip(angle, -Config.MAX_ANGLE, Config.MAX_ANGLE)
        return angle, int(lane_center), scan_y

    def get_smoothed_angle(self, target_angle: float) -> float:
        self.prev_angle = self.prev_angle * (1 - Config.SMOOTHING) + target_angle * Config.SMOOTHING
        return self.prev_angle

    def draw_overlays(self, frame: np.ndarray, mask: np.ndarray, angle: float) -> np.ndarray:
        final_frame = frame.copy()

        # 1. Ego-Lane Scanline Visualization
        ego_line = self._draw_scanline(final_frame.shape, mask)
        final_frame = cv2.addWeighted(final_frame, 1.0, ego_line, Config.SCAN_ALPHA, 0)

        # 2. Green Lane Overlay
        if mask.any():
            color_mask = np.zeros_like(final_frame, dtype=np.uint8)
            color_mask[mask > 0] = Config.LANE_COLOR_BGR
            mask_bool = mask > 0
            final_frame[mask_bool] = (
                final_frame[mask_bool].astype(np.float32) * (1 - Config.LANE_ALPHA) +
                color_mask[mask_bool].astype(np.float32) * Config.LANE_ALPHA
            ).astype(np.uint8)

        # 3. Steering Wheel
        final_frame = self._draw_steering_wheel(final_frame, angle)

        # 4. Text Info
        cv2.putText(final_frame, f"Angle: {angle:.1f}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        return final_frame

    def _draw_scanline(self, img_shape: Tuple, mask_pred: np.ndarray) -> np.ndarray:
        h, w = img_shape[:2]
        overlay = np.zeros(img_shape, dtype=np.uint8)
        bin_mask = (mask_pred > 0).astype(np.uint8)

        start_y = int(h * 0.45)
        prev_center = w * 0.5
        prev_width = None

        # Note: Vectorizing this specific tracking logic is complex due to the `prev_center` dependency.
        # Keeping loop for correctness as requested, but optimized variable access.
        for y in range(h - 1, start_y, -1):
            row_pixels = bin_mask[y, :]
            if not np.any(row_pixels): continue

            xs = np.where(row_pixels)[0]

            # Use prev_center to find closest lane boundaries
            l_cands = xs[xs < prev_center]
            r_cands = xs[xs >= prev_center]

            xl = l_cands.max() if l_cands.size > 0 else None
            xr = r_cands.min() if r_cands.size > 0 else None

            draw_l, draw_r = None, None

            if xl is not None and xr is not None and xl < xr:
                wd = xr - xl
                if 10 < wd < w * 0.9:
                    draw_l, draw_r = xl, xr
                    prev_width = wd
                    prev_center = (xl + xr) / 2
            elif prev_width is not None:
                if xl is not None:
                    draw_l = xl
                    draw_r = int(xl + prev_width)
                    prev_center = (draw_l + draw_r) / 2
                elif xr is not None:
                    draw_r = xr
                    draw_l = int(xr - prev_width)
                    prev_center = (draw_l + draw_r) / 2

            if draw_l is not None and draw_r is not None:
                # Optimized drawing: simple array slicing is faster than line drawing for horizontal lines
                draw_l = max(0, min(draw_l, w-1))
                draw_r = max(0, min(draw_r, w-1))
                overlay[y, int(draw_l):int(draw_r)] = Config.SCANLINE_COLOR

        return overlay

    def _draw_steering_wheel(self, frame: np.ndarray, angle: float) -> np.ndarray:
        rows, cols = frame.shape[:2]
        wh, ww = self.wheel_img.shape[:2]

        M = cv2.getRotationMatrix2D((ww // 2, wh // 2), -angle, 1.0)
        rotated = cv2.warpAffine(self.wheel_img, M, (ww, wh),
                                 flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)

        target_size = 300
        rotated = cv2.resize(rotated, (target_size, target_size))
        r_h, r_w = rotated.shape[:2]

        x_off = (cols - r_w) // 2
        y_off = rows - r_h - 20

        roi = frame[y_off:y_off+r_h, x_off:x_off+r_w]

        if rotated.shape[2] == 4:
            b, g, r, a = cv2.split(rotated)
            mask = (a.astype(np.float32) / 255.0)[..., None]
            roi[:] = (mask * cv2.merge((b, g, r)) + (1 - mask) * roi.astype(np.float32)).astype(np.uint8)
        else:
            roi[:] = rotated

        frame[y_off:y_off+r_h, x_off:x_off+r_w] = roi
        return frame

# ==========================================
# 4) Main Execution
# ==========================================
def main():
    # File Discovery
    raw_files = natsorted(glob.glob(os.path.join(Config.RAW_IMG_DIR, "*.jpg")))
    seg_files = natsorted(glob.glob(os.path.join(Config.SEG_VIS_DIR, "*.png")))

    if not seg_files:
        seg_files = natsorted(glob.glob(os.path.join(Config.SEG_VIS_DIR, "*.jpg")))

    if not raw_files: raise FileNotFoundError(f"No raw images in {Config.RAW_IMG_DIR}")
    if not seg_files: raise FileNotFoundError(f"No seg images in {Config.SEG_VIS_DIR}")

    min_len = min(len(raw_files), len(seg_files))
    print(f"[Info] Found {len(raw_files)} raw, {len(seg_files)} seg -> Processing {min_len} frames.")

    # Prepare Output
    os.makedirs(os.path.dirname(Config.OUT_VIDEO_PATH), exist_ok=True)
    first_frame = cv2.imread(raw_files[0])
    h, w = first_frame.shape[:2]

    writer = cv2.VideoWriter(Config.OUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*"mp4v"), Config.FPS, (w, h))

    # Initialize Pilot
    pilot = LanePilot()
    print(f"🎬 Processing started -> {Config.OUT_VIDEO_PATH}")

    # Process Loop
    for i, (r_path, s_path) in enumerate(zip(raw_files[:min_len], seg_files[:min_len]), 1):

        raw_frame = cv2.imread(r_path)
        seg_vis_frame = cv2.imread(s_path)

        if raw_frame is None or seg_vis_frame is None: continue

        # Resize viz frame if needed
        if seg_vis_frame.shape[:2] != (h, w):
            seg_vis_frame = cv2.resize(seg_vis_frame, (w, h))

        # 1. Inference
        mask = pilot.infer_mask(raw_frame)

        # 2. Control
        target_angle, _, _ = pilot.calculate_steering(mask)
        smooth_angle = pilot.get_smoothed_angle(target_angle)

        # 3. Visualize
        final_frame = pilot.draw_overlays(seg_vis_frame, mask, smooth_angle)

        writer.write(final_frame)

        if i % 10 == 0:
            print(f"Progress: {i}/{min_len} frames | Angle: {smooth_angle:.2f}°", end='\r')

    writer.release()
    print(f"\n✅ Done! Saved to: {Config.OUT_VIDEO_PATH}")

if __name__ == "__main__":
    main()

[Info] Found 246 raw, 246 seg -> Processing 246 frames.
[Init] Loading model from /content/drive/MyDrive/DataSet/best.pt...
🎬 Processing started -> /content/drive/MyDrive/DataSet/Autonomous_Steering.mp4

✅ Done! Saved to: /content/drive/MyDrive/DataSet/Autonomous_Steering.mp4
